# Helper functions
## Time Series Chart
## Converts a matplotlib figure to HTML compatible format

In [1]:
import matplotlib
matplotlib.use('TkAgg') 
import matplotlib.pyplot as plt
import base64
from io import BytesIO

# Provided helper functions
def generate_time_series_chart(cleaned_data, title='Time Series Data', x_label='Time', y_label='Value'):
    fig, ax = plt.subplots(figsize=(10,6))
    ax.plot(cleaned_data['Year'], cleaned_data['Value'], label=y_label, color='blue', marker='o')
    
    # Rotating and formatting date labels on x-axis
    plt.xticks(rotation=45)  # Rotate labels for better visibility
    
    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.grid(True)
    ax.legend()
    return fig


def get_image(fig):
    buffer = BytesIO()
    fig.savefig(buffer, format='png', bbox_inches='tight')
    buffer.seek(0)
    image_png = buffer.getvalue()
    buffer.close()
    graphic = base64.b64encode(image_png)
    graphic = graphic.decode('utf-8')
    return graphic

# Visualization of GDP and Inflation for specific countries (USA and CAN in this example)
# Assuming 'gdp_data' and 'inflation_data' are already fetched and prepared
def visualize_data(gdp_data, inflation_data, country_code):
    # Visualize GDP Data
    fig_gdp = generate_time_series_chart(
        cleaned_data=gdp_data[['Year', f'GDP_{country_code}']].rename(columns={f'GDP_{country_code}': 'Value'}),
        title=f'GDP over Time ({country_code})',
        x_label='Year',
        y_label='GDP'
    )
    gdp_image = get_image(fig_gdp)
    plt.show()  # Corrected to show all figures

    # Visualize Inflation Data
    fig_inflation = generate_time_series_chart(
        cleaned_data=inflation_data[['Year', f'Inflation_{country_code}']].rename(columns={f'Inflation_{country_code}': 'Value'}),
        title=f'Inflation Rate over Time ({country_code})',
        x_label='Year',
        y_label='Inflation Rate'
    )
    inflation_image = get_image(fig_inflation)
    plt.show()  # Corrected to show all figures



# Fetch Data
Using API from worldbank.org

In [2]:
import requests
import pandas as pd

def fetch_gdp(countries=['ALL']):
    all_gdp_data = pd.DataFrame()
    
    for country_code in countries:
        url = f"http://api.worldbank.org/v2/country/{country_code}/indicator/NY.GDP.MKTP.CD?format=json"
        response = requests.get(url)
        data = response.json()

        if len(data) > 1 and isinstance(data[1], list):  # Ensure there is data
            df = pd.DataFrame(data[1])
            df = df[['date', 'value']].rename(columns={'date': 'Year', 'value': f'GDP_{country_code}'})
            df = df.sort_values('Year', ascending=True)  # Sort by Year in ascending order
            all_gdp_data = pd.merge(all_gdp_data, df, on="Year", how="outer") if not all_gdp_data.empty else df

    return all_gdp_data

def fetch_inflation(countries=['ALL']):
    all_inflation_data = pd.DataFrame()
    
    for country_code in countries:
        url = f"http://api.worldbank.org/v2/country/{country_code}/indicator/FP.CPI.TOTL.ZG?format=json"
        response = requests.get(url)
        data = response.json()

        if len(data) > 1 and isinstance(data[1], list):  # Ensure there is data
            df = pd.DataFrame(data[1])
            df = df[['date', 'value']].rename(columns={'date': 'Year', 'value': f'Inflation_{country_code}'})
            df = df.sort_values('Year', ascending=True)  # Sort by Year in ascending order
            df = df.sort_values('Year', ascending=True)  # Sort by Year in ascending order

            all_inflation_data = pd.merge(all_inflation_data, df, on="Year", how="outer") if not all_inflation_data.empty else df

    return all_inflation_data





In [3]:
# Specify the countries you are interested in as a list of country codes
countries = ['USA', 'CAN']  # For example, United States and Canada

# Fetch and clean data for the specified countries
gdp_data = fetch_gdp(countries)
inflation_data = fetch_inflation(countries)


# Print the head (first few rows) of the data
print("GDP Data:")
print(gdp_data.head())  # Prints first 5 rows of the GDP data
print("\nInflation Data:")
print(inflation_data.head())  # Prints first 5 rows of the Inflation data


GDP Data:
   Year        GDP_USA       GDP_CAN
0  1973  1425376000000  1.317643e+11
1  1974  1545243000000  1.609492e+11
2  1975  1684904000000  1.744198e+11
3  1976  1873412000000  2.072716e+11
4  1977  2081826000000  2.123252e+11

Inflation Data:
   Year  Inflation_USA  Inflation_CAN
0  1973       6.177760       7.487647
1  1974      11.054805      10.997171
2  1975       9.143147      10.672189
3  1976       5.744813       7.541739
4  1977       6.501684       7.976445


In [4]:

# Call the function for specific countries
visualize_data(gdp_data, inflation_data, 'USA')
visualize_data(gdp_data, inflation_data, 'CAN')